In [1]:
import pickle
with open("ANDI_DATA/challenge_data.pkl", "rb") as f:
    challenge_data = pickle.load(f)


In [2]:
import numpy as np
length = []

for exp in range(12):
    for fov in range(30):
        for n in range(len(challenge_data[exp][fov])):
            length.append(len(challenge_data[exp][fov][0][n]))
            
print('max_length of data = {}'.format(np.max(length)))

max_length of data = 200


In [3]:
for exp in range(12):
    print(len(challenge_data[exp]))

30
30
30
30
30
30
30
30
30
30
30
30


In [4]:
def TMSD(traj, t_lags):
    ttt = np.zeros_like(t_lags, dtype= float)
    for idx, t in enumerate(t_lags): 
        for p in range(len(traj)-t):
            ttt[idx] += (traj[p]-traj[p+t])**2            
        ttt[idx] /= len(traj)-t    
    return ttt

def ta_msd(data_x,data_y):
    TA_MSD = []
    lag = len(data_x) 
    for i,s in enumerate(range(1,lag)):
        tamsd_s = []
        for t in range(len(data_x) - s):
            tamsd_s.append(((data_x[t+s] - data_x[t])**2) + ((data_y[t+s] - data_y[t])**2))
        TA_MSD.append(np.asarray(np.sum(tamsd_s)/len(tamsd_s)))

    return np.asarray(TA_MSD)

def correlation(x,y):
    
    T = int(len(x) - 2)
    
    cor = np.zeros(T)
    for t in range(T):
        dx_1 = x[t+1] - x[t]
        dx_2 = x[t+2] - x[t+1]
        
        dy_1 = y[t+1] - y[t]
        dy_2 = y[t+2] - y[t+1]
        
        cor[t] = (dx_1*dx_2 + dy_1*dy_2)
    
    return np.mean(cor)



def feature_v2(xtrace,ytrace):
    
    traces = np.sqrt(xtrace**2 + ytrace**2)
    
    mean = np.mean(traces)    
    ms = np.std(traces) 


   
    vtraces = np.diff(traces)
    
    v = np.abs(vtraces)
    vmean = np.mean(v) 
    vms = np.std(v) 

    
    corr = correlation(xtrace,ytrace)
#     r = np.abs(np.diff(np.sqrt(xtrace**2 + ytrace**2)))
    r = traces
    f1 = np.max(r) - np.min(r) 
    f2 = np.max(v) - np.min(v) 
    
       
    tlag = np.linspace(1, len(traces)-1, 9, dtype = 'int')
    
    msd = TMSD(traces, tlag)
    A = np.vstack([np.log(tlag), np.ones(len(np.log(tlag)))]).T
    nw1, nw0 = np.linalg.lstsq(A, np.log(msd), rcond=None)[0]


    feature = np.asarray([mean, ms, vmean, vms, np.exp(nw0)/4,f1,f2, corr]) 


    return feature


challenge_data = [exp][fov][x,y,z][index][t]

In [5]:
def remove_close_elements(input_list):
    i = 0
    while i < len(input_list) - 1:
        if input_list[i+1] - input_list[i] < 3:
            input_list.pop(i)
        else:
            i += 1
    return input_list

In [6]:
# load feature_data of challenge_data

with open('ANDI_DATA/feature_data (challenge_feature_v2_array).pkl', 'rb') as f:
    feature_data = pickle.load(f)
    
          

In [7]:
# calculate model input

from tensorflow import reduce_sum

def weighted_sum_data_k(
    data,
    coefficients: list = [0, 0, 1, 0, 1, 0, 0, 0]
):

    for idx, coefficient in enumerate(coefficients):
        data[..., idx] = data[..., idx] * coefficient

    data = reduce_sum(
        data,
        axis=len(data.shape) - 1,
        keepdims=True
    )

    return data

def weighted_sum_data_alpha(
    data,
    coefficients: list = [1, 1, 1, 1, 1, 1, 1, 1]
):

    for idx, coefficient in enumerate(coefficients):
        data[..., idx] = data[..., idx] * coefficient

    data = reduce_sum(
        data,
        axis=len(data.shape) - 1,
        keepdims=True
    )

    return data




test_data_alpha = {}
test_data_k = {}
for exp in range(12):
    print('{} calculating..'.format(exp))
    test_data_k_exp = {}
    for fov in range(30):
        with open('ANDI_DATA/feature_data (challenge_feature_v2_array).pkl', 'rb') as f:
            feature_data = pickle.load(f)
        test_data_k_exp[fov] =  weighted_sum_data_k(feature_data[exp][fov])
    test_data_k[exp] = test_data_k_exp
    
for exp in range(12):
    print('{} calculating..'.format(exp))
    test_data_alpha_exp = {}
    for fov in range(30):
        with open('ANDI_DATA/feature_data (challenge_feature_v2_array).pkl', 'rb') as f:
            feature_data = pickle.load(f)
        test_data_alpha_exp[fov] =  weighted_sum_data_alpha(feature_data[exp][fov])
    test_data_alpha[exp] = test_data_alpha_exp


0 calculating..
1 calculating..
2 calculating..
3 calculating..
4 calculating..
5 calculating..
6 calculating..
7 calculating..
8 calculating..
9 calculating..
10 calculating..
11 calculating..
0 calculating..
1 calculating..
2 calculating..
3 calculating..
4 calculating..
5 calculating..
6 calculating..
7 calculating..
8 calculating..
9 calculating..
10 calculating..
11 calculating..


In [8]:
with open('ANDI_DATA/alpha_input (challenge_feature_v2_array).pkl', 'wb') as f:
    pickle.dump(test_data_alpha, f)
    
with open('ANDI_DATA/k_input (challenge_feature_v2_array).pkl', 'wb') as f:
    pickle.dump(test_data_k, f)

In [9]:
with open('ANDI_DATA/alpha_input (challenge_feature_v2_array).pkl', 'rb') as f:
    alpha_input = pickle.load(f)
    
with open('ANDI_DATA/k_input (challenge_feature_v2_array).pkl', 'rb') as f:
    k_input = pickle.load(f)

In [12]:
print(len(alpha_input))
print(len(k_input))

12
12


In [13]:
# load model

from tensorflow import keras

model_alpha = keras.models.load_model('models/predict_alpha.hdf5')
model_k = keras.models.load_model('models/predict_k.hdf5')
    


In [14]:
def remove_close_elements(input_list):
    i = 0
    while i < len(input_list) - 1:
        if input_list[i+1] - input_list[i] < 3:
            input_list.pop(i)
        else:
            i += 1
    return input_list

In [15]:
# consider alpha and k

def cpd_v1_two_side(alpha,k,T,Th_alpha,Th_k, end_index):
    
    pred_cp = []
    reconstructed_feature1 = alpha[:int(T-end_index)]
    reconstructed_feature2 = k[:int(T-end_index)]
    T_length = T

    alpha_diff_max = np.max(np.abs(np.diff(reconstructed_feature1)))
    k_diff_max = np.max(np.abs(np.diff(reconstructed_feature2)))

    if alpha_diff_max > Th_alpha or k_diff_max > Th_k:
        if alpha_diff_max > k_diff_max:
            delta_feature = np.abs(np.diff(alpha))
            anomaly_index = 0
            anomaly_zip = []
            for n_zip in range(67):
                anomaly_zip.append([])


            for t, data_t in enumerate(delta_feature[:-1]):
                if data_t > Th_alpha:
                    anomaly_zip[anomaly_index].append((t,data_t))
                    if delta_feature[t + 1] <= Th_alpha:
                        anomaly_value = []
                        for a_index in range(len(anomaly_zip[anomaly_index])):
                            anomaly_value.append(anomaly_zip[anomaly_index][a_index][1])

                        max_anomaly_index = np.argmax(anomaly_value)
                        if (anomaly_zip[anomaly_index][max_anomaly_index][0] + 1) <= T:
                            pred_cp.append(anomaly_zip[anomaly_index][max_anomaly_index][0] + 1)

                        anomaly_index += 1
                        
        elif alpha_diff_max < k_diff_max:
            delta_feature = np.abs(np.diff(k))
            anomaly_index = 0
            anomaly_zip = []
            for n_zip in range(67):
                anomaly_zip.append([])


            for t, data_t in enumerate(delta_feature[:-1]):
                if data_t > Th_k:
                    anomaly_zip[anomaly_index].append((t,data_t))
                    if delta_feature[t + 1] <= Th_k:
                        anomaly_value = []
                        for a_index in range(len(anomaly_zip[anomaly_index])):
                            anomaly_value.append(anomaly_zip[anomaly_index][a_index][1])

                        max_anomaly_index = np.argmax(anomaly_value)
                        
                        if (anomaly_zip[anomaly_index][max_anomaly_index][0] + 1) <= T:
                            pred_cp.append(anomaly_zip[anomaly_index][max_anomaly_index][0] + 1)
                        anomaly_index += 1

        pred_cp.append(T)

    elif alpha_diff_max <= Th_alpha and k_diff_max <= Th_k:
        pred_cp.append(T)

    
    

    pred_cp = remove_close_elements(pred_cp)
    return pred_cp

In [16]:
# perform andi-challene2 tasks

def final_submit(feature_alpha,feature_k, model_alpha, model_k, data, Th_alpha, Th_k,end_index):

    predicted_n = []

    predicted_n.append(n)

    alpha_data = np.zeros((200))
    k_data = np.zeros((200))
            
    T_length = len(data)


                
    alpha = model_alpha.predict(feature_alpha)[0]
    k = model_k.predict(feature_k)[0]
        
    pred_cp = cpd_v1_two_side(alpha,k,T_length,Th_alpha,Th_k, end_index)

    if len(pred_cp) > 1:
        pred_cp.insert(0,0)
        for cp_num in range(int(len(pred_cp) - 1)):
            x = data[pred_cp[cp_num]: pred_cp[cp_num+1]] 
            y = data[pred_cp[cp_num]: pred_cp[cp_num+1]] 

            predicted_alpha = np.mean(alpha[int(pred_cp[cp_num]+1): int(pred_cp[cp_num+1]-1)])
            predicted_k = np.mean(k[int(pred_cp[cp_num]+1): int(pred_cp[cp_num+1]-1)])

            if predicted_alpha >= 1.9:
                model_label = 3

            elif predicted_alpha < 0.05 and predicted_k < 0.05:
                model_label = 0
                
            elif 0.95 <= predicted_alpha <= 1.05:
                model_label = 1

            else:
                model_label = 2


            predicted_n.append(predicted_k)
            predicted_n.append(predicted_alpha)
            predicted_n.append(model_label)
            predicted_n.append(pred_cp[cp_num+1])
                    
                    
    elif len(pred_cp) == 1:

        predict_cp = [0]
        predict_cp.append(pred_cp[0])


        for cp_num in range(int(len(predict_cp) - 1)):


            predicted_alpha = np.mean(alpha[int(predict_cp[cp_num]+1): int(predict_cp[cp_num+1]-1)])
            predicted_k = np.mean(k[int(predict_cp[cp_num]+1): int(predict_cp[cp_num+1]-1)])

  
            if predicted_alpha >= 1.9:
                model_label = 3
    
                        
            else:
                model_label = 2


            predicted_n.append(predicted_k)
            predicted_n.append(predicted_alpha)
            predicted_n.append(model_label)
            predicted_n.append(predict_cp[cp_num+1])
    return predicted_n

In [18]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow import keras


predicted_data = {}

Th_alpha = 0.06
Th_k = 0.06
end_index = 3


for exp in range(12):
    predicted_exp = {}
    for fov in range(30):
        predicted_fov = []
        print('{}, {} calculating..'.format(exp, fov))
        error_num1 = 0
        error_num2 = 0

        for n in range(len(challenge_data[exp][fov][0])):

            feature_alpha = tf.reshape(alpha_input[exp][fov][n],(1,1,191,1))
            feature_k = tf.reshape(k_input[exp][fov][n],(1,1,191,1))
                           
            data = challenge_data[exp][fov][0][n]

            predicted_n = final_submit(feature_alpha,feature_k, model_alpha, model_k, data, Th_alpha, Th_k,end_index)
            predicted_fov.append(predicted_n)
                    
            
            if predicted_n[-1] != len(challenge_data[exp][fov][0][n]):
                error_num1 += 1
            
            elif int(len(predicted_n)-1)%4 != 0:
                error_num2 += 1
                
            
        if error_num1 != 0:
            print('{} - error!'.format('not match data_length'))
            
        elif error_num2 != 0:
            print('{} - error!'.format('not match data_shape'))
  
            
        predicted_exp[fov] = predicted_fov
    predicted_data[exp] = predicted_exp
    
                

0, 0 calculating..
0, 1 calculating..
0, 2 calculating..
0, 3 calculating..
0, 4 calculating..
0, 5 calculating..
0, 6 calculating..
0, 7 calculating..
0, 8 calculating..
0, 9 calculating..
0, 10 calculating..
0, 11 calculating..
0, 12 calculating..
0, 13 calculating..
0, 14 calculating..
0, 15 calculating..
0, 16 calculating..
0, 17 calculating..
0, 18 calculating..
0, 19 calculating..
0, 20 calculating..
0, 21 calculating..
0, 22 calculating..
0, 23 calculating..
0, 24 calculating..
0, 25 calculating..
0, 26 calculating..
0, 27 calculating..
0, 28 calculating..
0, 29 calculating..
1, 0 calculating..
1, 1 calculating..
1, 2 calculating..
1, 3 calculating..
1, 4 calculating..
1, 5 calculating..
1, 6 calculating..
1, 7 calculating..
1, 8 calculating..
1, 9 calculating..
1, 10 calculating..
1, 11 calculating..
1, 12 calculating..
1, 13 calculating..
1, 14 calculating..
1, 15 calculating..
1, 16 calculating..
1, 17 calculating..
1, 18 calculating..
1, 19 calculating..
1, 20 calculating..


In [ ]:
# write predicted_data

for exp in range(len(challenge_data)):
    for fov in range(30):
   
        file_name = 'predict_data/exp_{}/fov_{}.txt'.format(exp,fov)

        text = '\n'.join([','.join(map(str, row)) for row in predicted_data[exp][fov]])

        with open(file_name, 'w') as f:
            f.write(text)